In [1]:
!pip install torch torchvision
%xmode Verbose

import numpy as np
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
torch.__version__

from torchvision import datasets
import torchvision.transforms as transforms

device = torch.device("cuda")

You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Exception reporting mode: Verbose


In [2]:
from torch.autograd import Variable
import torch.nn.functional as F

class SimpleCNN(torch.nn.Module):
  def __init__(self):
    super(SimpleCNN,self).__init__()
    
    self.conv1_out_ch = 64
    self.conv2_out_ch = 64
    self.conv3_out_ch = 64
    self.conv4_out_ch = 64
    self.conv5_out_ch = 64
    self.conv6_out_ch = 64

    self.dropout_p = 0.5
    
    self.conv1 = torch.nn.Conv2d(3, self.conv1_out_ch, kernel_size=3, stride=1, padding=1)
    self.bn1 = torch.nn.BatchNorm2d(self.conv1_out_ch)
    self.dropout1 = torch.nn.Dropout(p=self.dropout_p)
    self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
    
    self.conv2 = torch.nn.Conv2d(self.conv1_out_ch, self.conv2_out_ch, kernel_size=3, stride=1, padding=1)
    self.bn2 = torch.nn.BatchNorm2d(self.conv2_out_ch)
    self.dropout2 = torch.nn.Dropout(p=self.dropout_p)

    self.conv3 = torch.nn.Conv2d(self.conv2_out_ch, self.conv3_out_ch, kernel_size=3, stride=1, padding=1)
    self.bn3 = torch.nn.BatchNorm2d(self.conv3_out_ch)
    self.dropout3 = torch.nn.Dropout(p=self.dropout_p)

    self.conv4 = torch.nn.Conv2d(self.conv3_out_ch, self.conv4_out_ch, kernel_size=3, stride=1, padding=1)
    self.bn4 = torch.nn.BatchNorm2d(self.conv4_out_ch)
    self.dropout4 = torch.nn.Dropout(p=self.dropout_p)

    self.conv5 = torch.nn.Conv2d(self.conv4_out_ch, self.conv5_out_ch, kernel_size=3, stride=1, padding=1)
    self.bn5 = torch.nn.BatchNorm2d(self.conv5_out_ch)
    self.dropout5 = torch.nn.Dropout(p=self.dropout_p)

    self.conv6 = torch.nn.Conv2d(self.conv5_out_ch, self.conv6_out_ch, kernel_size=3, stride=1, padding=1)
    self.bn6 = torch.nn.BatchNorm2d(self.conv6_out_ch)
    self.dropout6 = torch.nn.Dropout(p=self.dropout_p)

    self.fc1 = torch.nn.Linear(self.conv6_out_ch*16*16, 64)
    self.fc2 = torch.nn.Linear(64, 10)
    
  def forward(self,x):
    x = F.relu(self.bn1(self.conv1(x)))
    x = self.dropout1(x)
    x = self.pool(x)

    x = F.relu(self.bn2(self.conv2(x)))
    x = self.dropout2(x)
    
    x = F.relu(self.bn3(self.conv3(x)))
    x = self.dropout3(x)
    
    x = F.relu(self.bn4(self.conv4(x)))
    x = self.dropout4(x)
    
    x = F.relu(self.bn5(self.conv5(x)))
    x = self.dropout5(x)
    
    x = F.relu(self.bn6(self.conv6(x)))
    x = self.dropout6(x)
    
    x = x.view(-1, self.conv6_out_ch*16*16)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return (x)

In [3]:
def eval_accuracy(loader, model) :
  y_true = []
  y_pred = []

  count = 0
  accuracy_sum = 0

  import sys

  with torch.no_grad() :
    for x, y in loader :
      x = x.float().to(device)
      y = y.to(device)
      model.eval()
      outputs = model.forward(x)

      outputs = outputs.cpu().numpy()
      y_pred = np.argmax(outputs, axis=1)
      y = y.cpu().numpy()

      match = (y == y_pred).astype('uint8')
      accuracy_sum += np.sum(match) 
      count += len(match)

    accuracy = accuracy_sum / count
    return accuracy

In [4]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = torchvision.datasets.CIFAR10(root='./cifardata', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./cifardata', train=False, download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

from torch.utils.data.sampler import SubsetRandomSampler

#Training
n_training_samples=45000
train_sampler=SubsetRandomSampler(np.arange(n_training_samples))

#Validation
n_val_samples = 5000
val_sampler = SubsetRandomSampler(np.arange(n_training_samples, n_training_samples + n_val_samples, dtype=np.int64))

#Test
n_test_samples = 10000
test_sampler = SubsetRandomSampler(np.arange(n_test_samples, dtype=np.int64))


Files already downloaded and verified
Files already downloaded and verified


In [5]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, sampler=test_sampler, num_workers=2)
val_loader = torch.utils.data.DataLoader(train_set, batch_size=128, sampler=val_sampler, num_workers=2)

In [6]:
import torch.optim as optim

import time

def trainNet(net, batch_size, n_epochs, learning_rate):
  print("==== HYPERPARAMETERS ====")
  print("batch_size=", batch_size)
  print("n_epochs=",n_epochs)
  print("learning_rate=",learning_rate)
  print("="*30)
  
  # Get training data
  train_loader = torch.utils.data.DataLoader(train_set, 
                                             batch_size=batch_size,
                                             sampler=train_sampler,
                                             num_workers=2)
  n_batches = len(train_loader)
  
  # Create loss and optimizer functions
  loss = torch.nn.CrossEntropyLoss()
  optimizer = optim.Adam(net.parameters(), lr=learning_rate)
  
  training_start_time = time.time()
  
  for epoch in range(n_epochs):
    running_loss = 0.0
    print_every = n_batches
    start_time = time.time()
    total_train_loss = 0
    
    for i, data in enumerate(train_loader,0):
      inputs, labels = data
      inputs, labels = Variable(inputs), Variable(labels)
      inputs = inputs.float().to(device)
      labels = labels.to(device)
      
      optimizer.zero_grad()
      
      net.train()
      outputs = net(inputs)
      loss_size = loss(outputs, labels)
      loss_size.backward()
      optimizer.step()
      
      running_loss += loss_size.data
      total_train_loss += loss_size.data
      
      if (i+1)%print_every == 0:
        print("Epoch {}, {:d}% \t train_loss: {:.2f} took {:.2f}s".format(
              epoch+1, int(100*(i+1)/n_batches), running_loss/print_every, time.time()-start_time))
        running_loss = 0.0
        start_time = time.time()
      
    # At the end of the epoch, do a pass on the validation set
    total_val_loss = 0
    for inputs, labels in val_loader:
      
      inputs, labels = Variable(inputs), Variable(labels)
      inputs = inputs.float().to(device)
      labels = labels.to(device)
      
      net.eval()
      val_outputs = net(inputs)
      val_loss_size = loss(val_outputs, labels)
      total_val_loss += val_loss_size.data
      
    print("Validation loss = {:.2f}".format(total_val_loss/len(val_loader)))
          
  print("Training finished, took {:.2f}s".format(time.time()-training_start_time))
  



In [7]:
CNN = SimpleCNN()
CNN = CNN.to(device)
trainNet(CNN, batch_size=32, n_epochs=50, learning_rate=0.001)

==== HYPERPARAMETERS ====
batch_size= 32
n_epochs= 50
learning_rate= 0.001
Epoch 1, 100% 	 train_loss: 1.51 took 10.16s
Validation loss = 1.56
Epoch 2, 100% 	 train_loss: 1.20 took 10.20s
Validation loss = 1.39
Epoch 3, 100% 	 train_loss: 1.07 took 10.34s
Validation loss = 1.07
Epoch 4, 100% 	 train_loss: 0.99 took 10.30s
Validation loss = 1.34
Epoch 5, 100% 	 train_loss: 0.93 took 10.24s
Validation loss = 1.02
Epoch 6, 100% 	 train_loss: 0.88 took 14.65s
Validation loss = 1.16
Epoch 7, 100% 	 train_loss: 0.84 took 15.64s
Validation loss = 0.99
Epoch 8, 100% 	 train_loss: 0.80 took 15.81s
Validation loss = 1.10
Epoch 9, 100% 	 train_loss: 0.77 took 16.18s
Validation loss = 1.01
Epoch 10, 100% 	 train_loss: 0.75 took 15.87s
Validation loss = 0.97
Epoch 11, 100% 	 train_loss: 0.73 took 15.91s
Validation loss = 1.04
Epoch 12, 100% 	 train_loss: 0.72 took 15.86s
Validation loss = 1.13
Epoch 13, 100% 	 train_loss: 0.69 took 15.90s
Validation loss = 1.06
Epoch 14, 100% 	 train_loss: 0.68 too

In [8]:
accuracy = eval_accuracy(test_loader,CNN)
print("accuracy={:.2f}%".format(accuracy*100))  

accuracy=76.72%
